# Example Notebook for Network Visualization

### Change working directory to this root

In [ ]:
import os
os.chdir("..")

### Imports

In [ ]:
import torch
import hydra
import sys

from IPython.display import clear_output
from models.modules.base import load_inner_model_state_dict
from models.tokenizers.char import CharLevelTokenizer
from models.modules.diffusion_llm import DiffusionLLMLightningModule

### Load model

In [ ]:
# Infer device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Current device is {device}")

# Load network
with hydra.initialize(version_base=None, config_path="../config"):
    cfg = hydra.compose(config_name="dllm")
cfg.learning.batch_size = 128
weights_path = "weights/dllm.ckpt"
model: DiffusionLLMLightningModule = hydra.utils.instantiate(cfg.module)
model = load_inner_model_state_dict(model, weights_path).to(device).eval()

### Load data loader

In [ ]:
data_module = hydra.utils.instantiate(cfg.data)
data_module.setup("val")
loader = data_module.val_dataloader()

### Load Tokenizer

In [ ]:
tokenizer = CharLevelTokenizer()

## Inference

### Perform inference

In [ ]:
TEXT = "What is the meaning of life?"
tokenized_text = tokenizer.encode(TEXT)
tokenized_text = torch.tensor(tokenized_text).unsqueeze(0)

idx = 0
with torch.no_grad():
        for generated_tokens in model.stream(init_tokens=tokenized_text, seq_len=256, vocab_size=66, temperature=0.3):
            generated_text = tokenizer.decode(generated_tokens.squeeze().cpu().numpy())
            idx += 1
            print(f"Cycle: {idx} --- >")
            print(generated_text)
